In [1]:
import sys
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import pickle
import re
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
import datasets
from datasets import Dataset

import warnings # выключение отображение warning-ов
import transformers

warnings.filterwarnings('ignore')
print(transformers.__version__)
torch.set_printoptions(profile="full")

C:\Users\User\anaconda3\envs\gpu\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


4.27.2


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !{sys.executable} -m pip install datasets

In [ ]:
# dataset_path = 'topics'
# topic_names = ['text']
# for x in os.listdir(dataset_path):
#     topic_names.append(x)
# topic_names

In [ ]:
# dataset = pd.DataFrame(columns=topic_names)
# dataset

,text,alone,america,angel,anger,animal,baby,beach,beautiful,beauty,...,trust,truth,war,warning,water,weather,wedding,winter,work,world


In [ ]:
# for topic in tqdm(os.listdir(dataset_path)):
#     for file in os.listdir(f'{dataset_path}/{topic}'):
#         with open(f'{dataset_path}/{topic}/{file}', encoding = 'utf-8') as f:
#             str = f.read()
#             index = dataset.index[dataset['text'] == str]
#             # print(index.size)
#             if index.size != 0:  #  если текст есть в датасете
#                 # print('not None')
#                 dataset[topic][index] = 1
#             else:   # если текст отсутствует в датасете
                
#                 dataset = dataset.append({'text': str, topic: 1}, ignore_index=True)
#                 # print(file)

100%|██████████| 144/144 [2:31:58<00:00, 63.32s/it] 


In [ ]:
# dataset

,text,alone,america,angel,anger,animal,baby,beach,beautiful,beauty,...,trust,truth,war,warning,water,weather,wedding,winter,work,world
0,loneliness\nhis childhood\nwarms him up\nlonel...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Sit in a chair\nalone\nPeople I thought were\n...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Tell me why it has to be this way\nwhy I must ...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"A debtor to mercy alone, of covenant mercy I s...",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A year has been passed since I left home\nAgai...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13732,"Wake not for the world-heard thunder,\nNor the...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
13733,"We may roam through this world, like a child a...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
13734,When you were born in this world\nEveryone lau...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
13735,"WORLD, take good notice, silver stars fading,\...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [ ]:
# dataset_fillna = dataset.fillna(0)

In [ ]:
# dataset

,text,passion,daughter,weather,animal,sea,innocence,change,angel,brother,...,freedom,dark,hate,music,warning,teacher,peace,chicago,butterfly,murder
0,"Your passion is permanently etched on my soul,...",1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"When man had ceased to utter his lament, A god...",1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"O Gentle Love, ungentle for thy deed, Thou mak...",1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,PASSION TOUCHED HER LIPS Passion touched her l...,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,You eclipse me & I have stained the Sun with b...,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13731,Violence is nobody s leaven. Murder will not g...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
13732,St. Patrick s Church was sparsely filled For F...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
13733,No one sleeps more beautifully than you. But I...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
13734,"Pastor in the pulpit, the Sanctuary, supposedl...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
# with open(r'dataset_fillna.pkl', 'wb') as f:
#     pickle.dump(dataset_fillna, f)

Загрузка датасета

In [ ]:
# with open(r'dataset_fillna.pkl', 'rb') as f:
#     dataset = pickle.load(f)

Обработка датасета

In [ ]:
# EXCLUDE_SYMBOLS_STR = u''.join(['№', '«', 'ђ', '°', '±', 'ћ', '‰', '»', 'ѓ', 'µ', '·', 'ґ', 'њ', 'ї', 'џ', 'є', '‹',
#                                 '‡', '†', '¶', 'ќ', '€', '“', 'ў', '§', '„', '”', '\ufeff', 'љ', '›', '•', '—', 
#                                 '\x7f', '\xad', '¤', '\xa0', '\u200e', 'Š', 'ō', 'ä', '™', '×', '\'', '~'])

# regex_symb = re.compile('[%s]' % re.escape(EXCLUDE_SYMBOLS_STR))

# for row in range(dataset.shape[0]):
#     dataset["text"][row] = regex_symb.sub(' ',  dataset["text"][row]) # убираем дополнительные символы
#     dataset["text"][row] = dataset["text"][row].replace("\n", " ") # убираем \n
#     dataset["text"][row] = re.sub(' +', ' ', dataset["text"][row]) # оставляем максимум 1 пробел
#     dataset["text"][row] = dataset["text"][row].strip() # удаляем лишние пробелы в начале и конце статей
    

In [ ]:
# with open(r'dataset_processed.pkl', 'wb') as f:
#     pickle.dump(dataset, f)

In [2]:
with open(r'dataset_processed.pkl', 'rb') as f:
# with open(r'/content/drive/MyDrive/Универ/dataset_processed.pkl', 'rb') as f:
    dataset = pickle.load(f)

In [3]:
dataset["text"][1]

'When man had ceased to utter his lament, A god then let me tell my tale of sorrow. WHAT hope of once more meeting is there now In the still-closed blossoms of this day? Both heaven and hell thrown open seest thou; What wav ring thoughts within the bosom play No longer doubt! Descending from the sky, She lifts thee in her arms to realms on high. And thus thou into Paradise wert brought, As worthy of a pure and endless life; Nothing was left, no wish, no hope, no thought, Here was the boundary of thine inmost strife: And seeing one so fair, so glorified, The fount of yearning tears was straightway dried. No motion stirr d the day s revolving wheel, In their own front the minutes seem d to go; The evening kiss, a true and binding seal, Ne er changing till the morrow s sunlight glow. The hours resembled sisters as they went. Yet each one from another different. The last hour s kiss, so sadly sweet, effac d A beauteous network of entwining love. Now on the threshold pause the feet, now has

In [4]:
dataset.drop("text", axis=1)

,passion,daughter,weather,animal,sea,innocence,change,angel,brother,world,...,freedom,dark,hate,music,warning,teacher,peace,chicago,butterfly,murder
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13731,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
13732,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
13733,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
13734,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


Разделение на датасет для тренировки и проверки

In [5]:
train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)

In [58]:
device = torch.device("cuda")

Topics

In [6]:
labels = dataset.columns[1:].values
labels

array(['passion', 'daughter', 'weather', 'animal', 'sea', 'innocence',
       'change', 'angel', 'brother', 'world', 'paris', 'work', 'soldier',
       'together', 'christmas', 'dance', 'sleep', 'happy', 'faith',
       'school', 'courage', 'loss', 'romance', 'nature', 'identity',
       'car', 'greed', 'lost', 'june', 'truth', 'destiny', 'trust',
       'children', 'believe', 'culture', 'silver', 'frog', 'red', 'food',
       'sun', 'romantic', 'time', 'lust', 'mother', 'heaven', 'memory',
       'crazy', 'money', 'racism', 'girl', 'home', 'evil', 'future',
       'running', 'carpe diem', 'sorrow', 'january', 'funeral', 'travel',
       'spring', 'fire', 'poem', 'suicide', 'remember', 'poetry',
       'football', 'green', 'laughter', 'cinderella', 'poverty', 'today',
       'birth', 'happiness', 'life', 'son', 'winter', 'hair', 'rose',
       'rainbow', 'god', 'star', 'house', 'power', 'sympathy',
       'depression', 'snake', 'night', 'sick', 'despair', 'husband',
       'rain', 'lov

In [7]:
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}

BERT tokenizer

In [8]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased") # create a tokenizer that corresponds to BERT.

Data preprocess

In [9]:
def preprocess_data(examples):
    # take a batch of texts
    text = examples["text"]
    # encode them
    encoding = tokenizer(text, padding="max_length", truncation=True, max_length=256)
    # add labels
    labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
    # create numpy array of shape (batch_size, num_labels)
    labels_matrix = np.zeros((len(text), len(labels)))
    # fill numpy array
    for idx, label in enumerate(labels):
        labels_matrix[:, idx] = labels_batch[label]

    encoding["labels"] = labels_matrix.tolist()

    return encoding

In [10]:
dd = datasets.DatasetDict({"train":Dataset.from_pandas(train_dataset),"test":Dataset.from_pandas(test_dataset)})
dd

DatasetDict({
    train: Dataset({
        features: ['text', 'passion', 'daughter', 'weather', 'animal', 'sea', 'innocence', 'change', 'angel', 'brother', 'world', 'paris', 'work', 'soldier', 'together', 'christmas', 'dance', 'sleep', 'happy', 'faith', 'school', 'courage', 'loss', 'romance', 'nature', 'identity', 'car', 'greed', 'lost', 'june', 'truth', 'destiny', 'trust', 'children', 'believe', 'culture', 'silver', 'frog', 'red', 'food', 'sun', 'romantic', 'time', 'lust', 'mother', 'heaven', 'memory', 'crazy', 'money', 'racism', 'girl', 'home', 'evil', 'future', 'running', 'carpe diem', 'sorrow', 'january', 'funeral', 'travel', 'spring', 'fire', 'poem', 'suicide', 'remember', 'poetry', 'football', 'green', 'laughter', 'cinderella', 'poverty', 'today', 'birth', 'happiness', 'life', 'son', 'winter', 'hair', 'rose', 'rainbow', 'god', 'star', 'house', 'power', 'sympathy', 'depression', 'snake', 'night', 'sick', 'despair', 'husband', 'rain', 'love', 'river', 'city', 'childhood', 'sister',

In [11]:
encoded_dataset = dd.map(preprocess_data, batched=True, remove_columns=dd['train'].column_names)

Пример

In [ ]:
example = encoded_dataset['train'][1]
# example = dd['train'][1]
print(example.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])


In [ ]:
example

{'input_ids': [101,
  1045,
  2156,
  1996,
  3548,
  10998,
  2006,
  4823,
  1037,
  3147,
  2162,
  2299,
  2006,
  1996,
  2346,
  2027,
  2907,
  3031,
  2037,
  4409,
  2004,
  1996,
  3554,
  3632,
  2006,
  2035,
  2154,
  2146,
  2954,
  1010,
  2233,
  1010,
  1998,
  2954,
  5607,
  1010,
  6614,
  1998,
  5607,
  2157,
  3251,
  1996,
  3103,
  12342,
  2015,
  1010,
  2023,
  2003,
  2162,
  2023,
  2003,
  2162,
  3251,
  1996,
  15811,
  10364,
  2015,
  2023,
  2003,
  2162,
  2073,
  2057,
  3102,
  2256,
  2814,
  1010,
  5208,
  1998,
  3428,
  2005,
  2027,
  2024,
  2256,
  6716,
  2005,
  2613,
  2065,
  2027,
  5454,
  2114,
  2149,
  2005,
  2178,
  2027,
  2175,
  2000,
  2162,
  2138,
  1997,
  1037,
  9210,
  1997,
  2576,
  2273,
  2040,
  3233,
  4142,
  1025,
  1999,
  14908,
  4822,
  2145,
  3173,
  2037,
  25636,
  1996,
  3548,
  2954,
  2127,
  2053,
  2028,
  4832,
  2039,
  4902,
  999,
  2954,
  1010,
  2233,
  1010,
  1998,
  2954,
  5607,
  1010,

In [ ]:
tokenizer.decode(example['input_ids'])

'[CLS] i see the soldiers marching on singing a cold war song on the road they hold onto their guns as the fighting goes on all day long fight, march, and fight shoot, aim and shoot right whether the sun shines, this is war this is war whether the rains pours this is war where we kill our friends, sisters and brothers for they are our enemies for real if they choose against us for another they go to war because of a handful of political men who stand alive ; in erect offices still holding their pens the soldiers fight until no one stands up anymore! fight, march, and fight shoot, aim and shoot right whether the sun shines, this is war this is war whether the rains pours www. sylviachidi. com [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

In [ ]:
example['labels']

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [ ]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

['war']

Изменение формата

In [12]:
encoded_dataset.set_format("torch")

# Model

In [13]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", 
                                                           problem_type="multi_label_classification", 
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

## Train the model!

We are going to train the model using HuggingFace's Trainer API. This requires us to define 2 things: 

* `TrainingArguments`, which specify training hyperparameters. All options can be found in the [docs](https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments). Below, we for example specify that we want to evaluate after every epoch of training, we would like to save the model every epoch, we set the learning rate, the batch size to use for training/evaluation, how many epochs to train for, and so on.
* a `Trainer` object (docs can be found [here](https://huggingface.co/transformers/main_classes/trainer.html#id1)).

В качестве метрики для оценивания будем использовать F1 score, так как ...

In [14]:
batch_size = 8 # размер батча

In [ ]:
# def compute_loss(self, model, inputs, return_outputs=False):
#     """
#     How the loss is computed by Trainer. By default, all models return the loss in the first element.
#     Subclass and override for custom behavior.
#     """
#     if self.label_smoother is not None and "labels" in inputs:
#         labels = inputs.pop("labels")
#     else:
#         labels = None
#     outputs = model(**inputs)
#     # Save past state if it exists
#     # TODO: this needs to be fixed and made cleaner later.
#     if self.args.past_index >= 0:
#         self._past = outputs[self.args.past_index]

#     if labels is not None:
#         if unwrap_model(model)._get_name() in MODEL_FOR_CAUSAL_LM_MAPPING_NAMES.values():
#             loss = self.label_smoother(outputs, labels, shift_labels=True)
#         else:
#             loss = self.label_smoother(outputs, labels)
#     else:
#         if isinstance(outputs, dict) and "loss" not in outputs:
#             raise ValueError(
#                 "The model did not return a loss from the inputs, only the following keys: "
#                 f"{','.join(outputs.keys())}. For reference, the inputs it received are {','.join(inputs.keys())}."
#             )
#         # We don't use .loss here since the model may return tuples instead of ModelOutput.
#         loss = outputs["loss"] if isinstance(outputs, dict) else outputs[0]

#     return (loss, outputs) if return_outputs else loss

In [64]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        print("inputs:", inputs)
    #     labels = inputs.get("labels")
        labels = inputs.pop("labels")

        outputs = model(**inputs)
        print("outputs:", outputs)

        # Save past state if it exists
        if self.args.past_index >= 0:
            self._past = outputs[self.args.past_index]

        logits = outputs.get("logits")

        
        # приводим предсказания к виду от 0 до 1
        sigmoid = torch.nn.Sigmoid()
        predictions = sigmoid(logits)

        # compute custom loss
        weights = []
        for batch in range(len(inputs["input_ids"])):
            weights.append([(1 + x*99) for x in labels[batch]])
    #     loss_fct = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 2.0, 3.0]))
        loss_fct = torch.nn.BCELoss(weight=torch.tensor(weights).to(device))
        loss = loss_fct(predictions, labels)
        return (loss, outputs) if return_outputs else loss

In [127]:
loss_fct = torch.nn.BCELoss()
# input = torch.randn(batch_size, 144, requires_grad=True)
input = torch.randint(0, 2, size=(batch_size, 144), dtype=torch.float, requires_grad=True)
print(input)
# target = torch.empty(3, dtype=torch.long).random_(5)
target = torch.randint(0, 2, size=(batch_size, 144), dtype=torch.float)
print(target)
output = loss_fct(input, target)
output
# output = loss_fct(input.view(-1, len(target[0])), labels.view(-1))

tensor([[1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 1.,
         1., 1., 0., 0., 0., 1., 1., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0.,
         1., 1., 1., 0., 0., 0., 1., 0., 0., 1., 1., 1., 0., 0., 1., 0., 1., 0.,
         1., 0., 1., 1., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.,
         1., 0., 1., 0., 1., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0.,
         0., 0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0.,
         1., 0., 0., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 0., 0.,
         0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 1., 1., 1.],
        [1., 1., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 0., 1., 1.,
         0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 1.,
         1., 1., 1., 1., 0., 0., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 1., 0.,
         1., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 0.,
         1., 0., 0., 1., 1.

tensor(50.9549, grad_fn=<BinaryCrossEntropyBackward0>)

In [16]:
args = TrainingArguments(
    f"bert-finetuned-sem_eval-english", # папка для сохранения контрольных точек
    # evaluation_strategy = "epoch", # оценивание во время обучения
    evaluation_strategy = "no",
    save_strategy = "epoch", # сохранение модели (после эпохи)
    learning_rate=2e-5, # lr для AdamW оптимизатора
    per_device_train_batch_size=batch_size, # размер батча при тренирвке
    per_device_eval_batch_size=batch_size, # размер батча при оценивании
    num_train_epochs=5, # число эпох обучения
    weight_decay=0.01, # вес для слоёв
    # load_best_model_at_end=True, # загружать ли лучшую модель
    metric_for_best_model="f1", # метрика, на основе которой выберется лучшая модель
    dataloader_drop_last=True
)

We are also going to compute metrics while training. For this, we need to define a compute_metrics function, that returns a dictionary with the desired metric values.

In [17]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from transformers import EvalPrediction
import torch
    
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions)) # переводим вероятности в область от 0 до 1

    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1 # если вероятность > treshold, то ставим 1 в предсказании (остальное 0)

    y_true = labels # реальное значение топиков
    
    print("probs:", probs, "shape:", probs.shape)
    print("y_true:", y_true, "shape:", y_true.shape)
    print("y_pred:", y_pred, "shape:", y_pred.shape)
    
    f1_micro_average, accuracy, precision, recall = 0, 0, 0, 0

    for elem in range(len(y_true)):
        f1_micro_average += f1_score(y_true[elem], y_pred[elem], average='micro')
        accuracy += accuracy_score(y_true[elem], y_pred[elem])
        precision += precision_score(y_true[elem], y_pred[elem], average='micro')
        recall += recall_score(y_true[elem], y_pred[elem], average='micro')

    metrics = {'f1': f1_micro_average/len(y_true),
               'accuracy': accuracy/len(y_true),
               'precision': precision/len(y_true),
               'recall': recall/len(y_true)
              }

    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    # print("p.predictions:", p.predictions)
    print("preds:", preds, "shape:", preds.shape)
    
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

In [ ]:
# encoded_dataset['train'][0]['labels'].type()

In [18]:
encoded_dataset['train']['input_ids'][1]

tensor([  101,  1045,  2156,  1996,  3548, 10998,  2006,  4823,  1037,  3147,
         2162,  2299,  2006,  1996,  2346,  2027,  2907,  3031,  2037,  4409,
         2004,  1996,  3554,  3632,  2006,  2035,  2154,  2146,  2954,  1010,
         2233,  1010,  1998,  2954,  5607,  1010,  6614,  1998,  5607,  2157,
         3251,  1996,  3103, 12342,  2015,  1010,  2023,  2003,  2162,  2023,
         2003,  2162,  3251,  1996, 15811, 10364,  2015,  2023,  2003,  2162,
         2073,  2057,  3102,  2256,  2814,  1010,  5208,  1998,  3428,  2005,
         2027,  2024,  2256,  6716,  2005,  2613,  2065,  2027,  5454,  2114,
         2149,  2005,  2178,  2027,  2175,  2000,  2162,  2138,  1997,  1037,
         9210,  1997,  2576,  2273,  2040,  3233,  4142,  1025,  1999, 14908,
         4822,  2145,  3173,  2037, 25636,  1996,  3548,  2954,  2127,  2053,
         2028,  4832,  2039,  4902,   999,  2954,  1010,  2233,  1010,  1998,
         2954,  5607,  1010,  6614,  1998,  5607,  2157,  3251, 

# Training

In [65]:
trainer = CustomTrainer(
    model.to(device), # модель для обучения
    args, # аргументы для обучения
    train_dataset=encoded_dataset["train"], # датасет, используемый при обучении
    eval_dataset=encoded_dataset["test"], # датасет, используемый для оценивания
    tokenizer=tokenizer, # токенизатор
    compute_metrics=compute_metrics # функция для подсчёта метрик при оценивании
)
# trainer.compute_loss = CustomTrainer.compute_loss

In [ ]:
trainer.train()

inputs: {'input_ids': tensor([[  101,  2637,   999,  2643,  2435,  2017,  5041,  3993, 16805,  4351,
          4020,  1010, 12467,  1010, 19817,  7361,  6558,  1012,  2007,  5761,
          1997, 22824,  2665, 24813,  2000,  2562,  1996,  5763,  2455,  1010,
          4760,  1012,  2637,   999,  2643,  2435,  2017,  3376,  2336,  2367,
          3837,  1010,  5636,  6447,  1010,  8578,  1012,  2007,  8150,  2015,
          1997,  2580,  6087,  2000,  2562,  1996, 13721,  8962,  1010, 13053,
          1012,  2637,   999,  2643,  2435,  2017,  8945, 16671, 18424, 11203,
          2015, 10059, 21569,  1010,  2878, 10500,  1010,  8982,  1012,  2007,
          3674, 10281,  1997,  7965, 16994,  6961,  2000,  2562,  1996,  6501,
          1998,  6861,  1010,  8577,  1012,  2637,   999,  2643,  2435,  2017,
         10190,  3993,  3716, 29454, 29206,  2102,  5082,  1999,  2974,  1012,
          2007,  5821,  2015,  1997,  3284,  4803,  2015,  2000,  2562,  1996,
          2137,  3959,  1010, 

outputs: SequenceClassifierOutput(loss=None, logits=tensor([[ 5.1029e-01,  3.2679e-02,  3.5592e-01,  2.6627e-01,  7.9680e-02,
         -7.9098e-02,  7.4592e-01, -1.1554e-01, -2.9521e-01, -7.6183e-03,
          5.5754e-02,  4.6268e-02,  4.8023e-01, -6.3855e-02, -2.5401e-01,
          5.3902e-01,  4.7783e-02, -1.1532e-01,  3.8793e-01, -1.8939e-03,
         -5.0855e-01, -1.2961e-01, -1.0391e-01,  2.2649e-01,  2.2444e-01,
          6.9458e-02,  7.4583e-01,  2.8169e-04, -2.9999e-01,  1.0377e-01,
          2.1494e-01, -1.0092e-01,  4.1097e-01,  3.7192e-02, -2.9920e-01,
          1.2995e-01, -1.9152e-01, -2.4384e-02, -1.4007e-01,  3.4642e-01,
         -1.0696e-01, -2.3940e-01,  3.0910e-01,  2.2218e-01, -2.5553e-01,
         -7.3588e-01, -2.3729e-01,  9.8818e-02, -2.1629e-01, -6.3358e-01,
          6.4746e-02,  8.6274e-01, -1.3459e-02,  5.5521e-01, -7.8252e-02,
          8.5312e-01,  2.0401e-01, -7.9377e-01, -2.8107e-01,  3.5373e-01,
          1.2936e-01, -2.3897e-02, -6.2592e-02, -6.1109e-01,

Step,Training Loss


inputs: {'input_ids': tensor([[  101,  1996,  8251,  1997,  2010,  2303,  6223,  2083,  2010,  3543,
          1010,  4372, 21508,  2033,  1999,  2010,  5304,  1012,  1045,  2298,
          2784,  2000,  4608,  1037, 12185,  1997,  2010,  3969,  1012,  5683,
          2002,  2038,  2525,  2464,  1998,  2579,  6664,  1997,  3067,  1012,
          1045,  4392,  1999,  2010,  3052,  1010,  1037,  4086, 24816, 17657,
          2026,  2303,  1012, 14899,  1999,  2000,  2032,  1010,  9461,  7806,
          1529,  1529,  1012,  2059, 10520,  1012,  2027,  2481,  1521,  1056,
          2674,  2026, 12098, 26797,  2099,  1010,  2134,  1521,  1056,  3305,
          2009,  1012,  6257,  1045,  2018, 24664, 19225, 21662,  1010,  2042,
          2026, 23199,  2869,  1010,  2371,  2569,  1010,  1996, 25466,  2015,
          1012,  2002,  2987,  1521,  1056,  2113,   999,  2002,  2001,  2569,
          1012,  2106, 23199,  2033,  1010, 16152,  2033,  2146,  3283,  1529,
          1529,  1996,  2034, 

outputs: SequenceClassifierOutput(loss=None, logits=tensor([[ 8.4974e-01, -8.8782e-02,  5.1807e-01, -1.6549e-02,  9.8645e-02,
         -9.5783e-03,  4.3498e-01, -1.0413e-01, -5.4597e-01,  1.0843e-02,
          4.9661e-02, -1.9871e-01,  9.3128e-02, -5.5588e-02, -5.0198e-01,
          5.8114e-01, -5.3737e-02, -1.4462e-02,  2.9296e-01, -8.0093e-03,
         -4.0270e-01, -1.3068e-01,  1.8561e-01,  3.6924e-01,  1.9742e-01,
         -7.0519e-02,  4.6422e-01,  5.8954e-02, -1.4652e-01, -7.0252e-02,
          5.5368e-01, -2.7393e-01,  3.1305e-02, -9.3843e-03, -2.4860e-01,
         -4.9292e-01,  1.1084e-01, -2.8612e-01, -8.0531e-02,  1.0555e-01,
         -3.1510e-01, -8.6842e-02,  1.5668e-01, -1.5814e-01,  5.6116e-02,
         -4.7989e-01, -1.3791e-02,  3.5650e-01, -3.2547e-01, -4.3105e-01,
          4.9730e-01,  4.1113e-01, -1.5362e-01,  5.9635e-01, -2.8358e-01,
          5.4793e-01, -2.6112e-01, -2.8832e-01, -7.6434e-02,  7.1343e-01,
          2.6492e-01, -2.1967e-01, -2.0473e-02, -6.7285e-01,

...

inputs: {'input_ids': tensor([[  101,  2011,  2010,  2217,  1010,  3005,  2420,  2024,  2627,  1010,
          3913,  6812,  1998, 29049,   999,  1998,  2010,  2159,  2008,  6237,
         12943, 14949,  2102,  2485,  1010,  2007,  1037, 13277,  1012,  2643,
          4496,  2158,  2013,  2331,  1010,  2012,  2197,  1010,  2293,  2089,
          8116,  1012,  2295,  1010,  1997,  2214,  1010,  2057, 18152,  1010,
          2026,  6203,  1010,  2331,  2323,  2025,  2202,  2032,  1025,  9587,
         14287,  2025, 15223,  2008,  2057,  2442,  2182, 24745,  2005,  3736,
          3489,  2032,  1025,  8328,  2682,  2010,  6542,  2053,  7697,  1010,
          4000,  2097,  2025,  5256,  2032,  1012,  2452,  3593,  4682,  2705,
          3147,  1998,  2757,  1010,  3092,  2010,  3909,  1010,  5122,  2010,
          2970,  1010,  2320, 26851,  1011,  2417,  1010,  9170,  2001,  2010,
          5996,  1012,  2173,  1037,  2962,  2682,  2010,  2132,  1010,  2735,
          2185,  1010, 19381, 

outputs: SequenceClassifierOutput(loss=None, logits=tensor([[-3.8810e-01,  1.6418e-01, -2.2846e-01,  9.6871e-02, -8.8983e-02,
          2.4934e-01,  6.1621e-02,  4.9624e-01,  3.0248e-01, -2.0253e-01,
          2.0587e-01, -1.3383e-01,  5.1646e-01, -2.2021e-02,  1.2995e-01,
          5.2023e-01,  8.6585e-02,  7.5757e-02, -4.1831e-01,  1.3763e-01,
          3.6913e-01,  1.3970e-01,  2.8891e-01, -1.6940e-01,  1.0326e-01,
          1.8898e-01,  5.3983e-01,  2.0483e-01, -2.1526e-01,  1.3273e-01,
          4.6087e-02,  3.1559e-01,  6.6941e-01,  4.5456e-01,  9.8840e-02,
          3.8844e-01,  1.0720e-01,  6.8663e-02, -1.9429e-01, -1.8773e-01,
          3.1922e-01,  3.3244e-01, -9.3256e-03,  1.7527e-01, -7.1957e-01,
          5.7175e-02, -8.2687e-02,  1.9478e-01,  2.7758e-01, -2.0106e-01,
         -1.2010e-01,  1.0966e-01,  1.6291e-01,  7.0260e-02,  4.1837e-01,
          2.7731e-02,  2.4649e-01, -2.1310e-01, -3.2008e-01,  1.8542e-01,
         -6.7231e-01, -9.7635e-02, -2.8673e-01,  2.0046e-01,

inputs: {'input_ids': tensor([[  101,  2166,  2003,  2066,  6565,  4153,  1010,  2440,  1997, 14038,
          1998,  3255,  1010,  6569,  2625,  1998,  4737,  2364,  1010,  2025,
          2000,  6065,  3246,  1998,  2943, 12452,  1010,  2000,  4553,  1037,
         10800,  2013,  2169,  4945,  1010,  4180,  2007,  2625,  1998,  2498,
          2000,  4013, 23887,  1010,  8190,  2062,  2021, 11752,  2000, 18094,
          1010,  2053,  6234,  2765,  2021,  3112,  2003,  2469,  1010,  2065,
          2166,  2001,  2412,  2000,  2022,  5744,  1010,  2040,  2052, 11839,
          1998,  2202, 11292,  1029,  4521,  2062,  1998,  4287,  1996,  3635,
          2614,  3637,  1998,  2131,  2039,  2397,  2166,  1998,  2331,  2024,
          2048,  3033,  1997,  5217,  4945,  1997,  2151,  2028,  2089,  3084,
          2149,  2000,  2514,  2166,  2089,  2693,  2006,  1998,  2987,  1521,
          1056,  2272,  2000,  2019,  2203, 11839,  2005,  3112,  1998,  3971,
          2000,  2273,  2094, 

outputs: SequenceClassifierOutput(loss=None, logits=tensor([[-1.8844e-01,  1.3078e-01, -4.3456e-01,  8.2139e-02,  8.5492e-02,
          3.9808e-01, -8.3444e-02,  3.5891e-01,  2.3267e-02, -1.6105e-02,
          3.1051e-01, -3.3173e-02,  2.0778e-02,  6.2243e-02,  1.7274e-01,
          1.3480e-01, -2.0435e-02,  1.2094e-01, -5.4849e-01,  3.7882e-01,
          3.5430e-01,  2.7066e-01,  4.4838e-01, -1.1565e-01, -1.4129e-01,
          4.1429e-01, -1.0474e-02,  2.9638e-03, -1.3687e-01,  7.2486e-02,
          2.3840e-02,  3.9860e-02,  5.6883e-01,  2.2880e-01,  4.7591e-01,
          1.3768e-01,  1.7764e-01, -3.0712e-02,  7.5455e-02, -4.6110e-01,
          3.6744e-01,  3.7173e-01,  4.6187e-02, -2.3838e-01, -3.8720e-01,
          6.0267e-01, -5.4605e-02, -3.7077e-02,  5.7464e-01, -1.6081e-01,
         -3.2093e-01, -4.0733e-01, -2.8793e-02, -2.2905e-01,  5.8513e-01,
         -2.6833e-01,  3.2453e-01,  4.4640e-01, -3.1933e-01, -6.1629e-02,
         -4.4901e-01, -6.5941e-02,  3.1397e-02,  5.8577e-01,

inputs: 

# Оценивание (вывод метрик)

In [22]:
trainer.evaluate()

preds: [[-4.9785857 -4.9183545 -4.6610045 ... -4.564983  -4.702362  -5.125838 ]
 [-5.120772  -4.949691  -4.822728  ... -4.821629  -4.841216  -5.080791 ]
 [-5.1487784 -5.198499  -5.0735016 ... -5.207233  -5.1089225 -5.0024867]
 ...
 [-4.905149  -4.7600746 -4.5454454 ... -4.395849  -4.6614575 -5.1463695]
 [-5.107263  -4.963121  -4.807657  ... -4.754567  -4.8438354 -5.0863786]
 [-5.065475  -4.876777  -4.7447047 ... -4.6627603 -4.7555447 -5.1208167]] shape: (2744, 144)
probs: tensor([[0.0068, 0.0073, 0.0094,  ..., 0.0103, 0.0090, 0.0059],
        [0.0059, 0.0070, 0.0080,  ..., 0.0080, 0.0078, 0.0062],
        [0.0058, 0.0055, 0.0062,  ..., 0.0054, 0.0060, 0.0067],
        ...,
        [0.0074, 0.0085, 0.0105,  ..., 0.0122, 0.0094, 0.0058],
        [0.0060, 0.0069, 0.0081,  ..., 0.0085, 0.0078, 0.0061],
        [0.0063, 0.0076, 0.0086,  ..., 0.0094, 0.0085, 0.0059]]) shape: torch.Size([2744, 144])
y_true: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0

{'eval_loss': 0.042169664055109024,
 'eval_f1': 0.99279741658574,
 'eval_accuracy': 0.99279741658574,
 'eval_precision': 0.99279741658574,
 'eval_recall': 0.99279741658574,
 'eval_runtime': 139.3971,
 'eval_samples_per_second': 19.713,
 'eval_steps_per_second': 2.468,
 'epoch': 5.0}

# Проверка работоспособности

In [46]:
old_func = trainer.compute_loss

In [55]:
trainer.compute_loss = CustomTrainer.compute_loss

In [56]:
trainer.compute_loss

<function __main__.CustomTrainer.compute_loss(self, model, inputs, return_outputs=False)>

In [57]:
old_func

<bound method Trainer.compute_loss of <transformers.trainer.Trainer object at 0x000002521C3F2730>>

In [38]:
dataset["text"][0]

'Your passion is permanently etched on my soul, On all flesh, branding skin north and south. Even after I have finished brushing my teeth I can relive your taste in my mouth. Your poems play new chords on the strings of my heart, The mark your smile leaves? I m still guessing! But the magical way you ve touched me from the start Poetry has no way of expressing. Though it s true that we have never met in the flesh And no photograph I ve ever seen. Could it be a connection from many past lives? Just the thought of you makes my world green. The best artist could never capture the beauty Of the truth you have let my heart see, Only one thing could possibly soften this blow: To know that in fact - you are for me.'

In [59]:
# text = "I'm happy I can finally train a model for multi-label classification"
text = dataset["text"][0]

encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}
outputs = trainer.model(**encoding)
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[-5.0030, -5.1958, -5.1621, -4.9543, -5.2506, -5.1825, -5.0038, -5.1347,
         -4.8827, -5.0914, -5.1783, -5.1049, -5.1394, -5.1198, -5.0976, -5.1077,
         -5.1914, -5.1099, -4.9030, -4.8808, -5.1730, -5.0736, -5.0172, -5.2330,
         -4.9303, -4.9446, -4.9266, -5.1816, -5.0068, -5.0266, -5.0413, -5.0181,
         -4.9399, -4.8454, -4.9312, -5.0202, -5.0345, -5.2027, -5.0776, -5.0303,
         -5.1469, -5.2382, -4.8653, -5.0807, -5.0542, -5.0405, -4.9018, -4.9499,
         -4.9938, -5.0895, -5.1750, -5.0889, -5.0454, -4.9495, -5.2309, -5.0935,
         -5.0749, -4.9713, -5.0134, -5.2493, -5.1911, -4.9994, -4.9540, -5.2577,
         -5.0240, -4.8649, -4.9963, -5.3320, -5.0180, -5.0681, -5.0632, -5.1000,
         -4.9871, -5.0886, -5.0265, -5.0950, -5.0028, -5.0622, -4.8744, -5.0540,
         -5.1874, -5.1897, -5.1270, -4.8600, -4.9199, -5.0025, -5.1140, -5.0965,
         -5.0811, -5.0007, -5.1578, -5.0179, -5.0207, -5.0032, -4.

The logits that come out of the model are of shape (batch_size, num_labels). As we are only forwarding a single sentence through the model, the batch_size equals 1. The logits is a tensor that contains the (unnormalized) scores for every individual label.

In [60]:
logits = outputs.logits
logits.shape

torch.Size([1, 144])

In [61]:
logits

tensor([[-5.0030, -5.1958, -5.1621, -4.9543, -5.2506, -5.1825, -5.0038, -5.1347,
         -4.8827, -5.0914, -5.1783, -5.1049, -5.1394, -5.1198, -5.0976, -5.1077,
         -5.1914, -5.1099, -4.9030, -4.8808, -5.1730, -5.0736, -5.0172, -5.2330,
         -4.9303, -4.9446, -4.9266, -5.1816, -5.0068, -5.0266, -5.0413, -5.0181,
         -4.9399, -4.8454, -4.9312, -5.0202, -5.0345, -5.2027, -5.0776, -5.0303,
         -5.1469, -5.2382, -4.8653, -5.0807, -5.0542, -5.0405, -4.9018, -4.9499,
         -4.9938, -5.0895, -5.1750, -5.0889, -5.0454, -4.9495, -5.2309, -5.0935,
         -5.0749, -4.9713, -5.0134, -5.2493, -5.1911, -4.9994, -4.9540, -5.2577,
         -5.0240, -4.8649, -4.9963, -5.3320, -5.0180, -5.0681, -5.0632, -5.1000,
         -4.9871, -5.0886, -5.0265, -5.0950, -5.0028, -5.0622, -4.8744, -5.0540,
         -5.1874, -5.1897, -5.1270, -4.8600, -4.9199, -5.0025, -5.1140, -5.0965,
         -5.0811, -5.0007, -5.1578, -5.0179, -5.0207, -5.0032, -4.9715, -5.1025,
         -5.1661, -5.0159, -

To turn them into actual predicted labels, we first apply a sigmoid function independently to every score, such that every score is turned into a number between 0 and 1, that can be interpreted as a "probability" for how certain the model is that a given class belongs to the input text.

Next, we use a threshold (typically, 0.5) to turn every probability into either a 1 (which means, we predict the label for the given example) or a 0 (which means, we don't predict the label for the given example).

In [62]:
# apply sigmoid + threshold
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())
print(probs)
predictions = np.zeros(probs.shape)
predictions[np.where(probs >= 0.5)] = 1
# turn predicted id's into actual label names
predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
print(predicted_labels)

tensor([0.0067, 0.0055, 0.0057, 0.0070, 0.0052, 0.0056, 0.0067, 0.0059, 0.0075,
        0.0061, 0.0056, 0.0060, 0.0058, 0.0059, 0.0061, 0.0060, 0.0055, 0.0060,
        0.0074, 0.0075, 0.0056, 0.0062, 0.0066, 0.0053, 0.0072, 0.0071, 0.0072,
        0.0056, 0.0066, 0.0065, 0.0064, 0.0066, 0.0071, 0.0078, 0.0072, 0.0066,
        0.0065, 0.0055, 0.0062, 0.0065, 0.0058, 0.0053, 0.0077, 0.0062, 0.0063,
        0.0064, 0.0074, 0.0070, 0.0067, 0.0061, 0.0056, 0.0061, 0.0064, 0.0070,
        0.0053, 0.0061, 0.0062, 0.0069, 0.0066, 0.0052, 0.0055, 0.0067, 0.0070,
        0.0052, 0.0065, 0.0077, 0.0067, 0.0048, 0.0066, 0.0063, 0.0063, 0.0061,
        0.0068, 0.0061, 0.0065, 0.0061, 0.0067, 0.0063, 0.0076, 0.0063, 0.0056,
        0.0055, 0.0059, 0.0077, 0.0072, 0.0067, 0.0060, 0.0061, 0.0062, 0.0067,
        0.0057, 0.0066, 0.0066, 0.0067, 0.0069, 0.0060, 0.0057, 0.0066, 0.0065,
        0.0060, 0.0057, 0.0073, 0.0059, 0.0067, 0.0065, 0.0079, 0.0071, 0.0070,
        0.0056, 0.0063, 0.0048, 0.0067, 